In [5]:
import ftplib
f = ftplib.FTP("10.*.*.*")  # 实例化FTP对象
f.login("***", "***")  # 登录
def ftp_download(remotename,localname):
    file_remote = "tiku/images/"+remotename
    file_local = "/data/fjs/data/math/"+localname
    bufsize = 1024  # 设置缓冲器大小
    fp = open(file_local, 'wb')
    f.retrbinary('RETR %s' % file_remote, fp.write, bufsize)
    fp.close()
remotename="upimages/images/part/8730D.png"
localname="8730D.png"
ftp_download(remotename,localname)
f.quit()

'221 Goodbye.'

In [58]:
import pymysql
from html.parser import HTMLParser
import re
import string
import json

#html解析
class mathHTMLParser(HTMLParser): 
    def __init__(self): 
        HTMLParser.__init__(self) 
        self.texts = [] #非结构化-文本
        self.images = [] #非结构化-图片
        #self.htmls = [] #半结构化-html格式
        #self.latexs = [] #半结构化-latex格式
        
    def handle_starttag(self, tag, attrs):
        if tag=='img':
            for a in attrs:
                if a[0] == 'src':
                    self.images.append(a[1])
        if tag=='div':
            for a in attrs:
                if a[0]=='style':
                    if a[1].find('url')!=-1:
                        self.images.append(a[1].split("'")[1])

    def handle_endtag(self, tag):
        pass

    def handle_data(self, data):
        data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
        if data.count('\n') == 0 and data!='':
            self.texts.append(data)
            
#mysql -h10.21.3.23 -ufangjiansheng -pfjs2019@cvte  CNK12
conn = pymysql.connect(host='10.*.*.*', user='***', passwd="***", db='CNK12')
cur = conn.cursor()
#数学学科小学学段913851条，初中1202389，高中1507119
items=[]#list，保存解析后的条目
for pagesize in range(0,18):#提取小学学段90万条数学习题，mysql性能低，采用limit分布查询
    sel_sql ="select distinct(a.question_uid),a.question_diff,a.question_knowledge,\
                 a.question_text,a.question_options,a.question_answer,a.question_explain,a.question_parse \
                 from cnk_question a \
                 left join cnk_knowledge_question b on (a.question_uid=b.question_uid) \
                 left join cnk_knowledge c on (b.knowledge_uid=c.knowledge_uid and c.subject_id=2 and c.pharse_id=1)\
                 limit %d,50000"%(pagesize*50000)
    cur.execute(sel_sql)
    results = cur.fetchall()
    for row in results:
        mhp = mathHTMLParser() 
        item={}#dict
        item['quid']=row[0]
        item['qdiff']=row[1]
        item['qklg']=row[2]
        mhp.feed(row[3])#题干
        mhp.feed(row[4])#选项
        mhp.feed(row[5])#答案
        mhp.feed(row[6])#解释
        mhp.feed(row[7])#分析
        item['qtext']=mhp.texts  #非结构化-文本
        item['qimage']=mhp.images #非结构化-图片
        #item['qhtml'] =mhp.htmls #半结构化-html格式
        #item['qlatex']=mhp.latexs #半结构化-latex格式
        mhp.close() 
        items.append(item)#插入list
    print ("page %d parsing completed."% pagesize)
cur.close()
conn.close()
print (len(items))#总长度
with open("/data/fjs/data/math/math.json","w") as dump_f:
    json.dump(items,dump_f)#保存到json文件
    print("Json file saving completed.")

page 0 parsing completed.
page 1 parsing completed.
page 2 parsing completed.
page 3 parsing completed.
page 4 parsing completed.
page 5 parsing completed.
page 6 parsing completed.
page 7 parsing completed.
page 8 parsing completed.
page 9 parsing completed.
page 10 parsing completed.
page 11 parsing completed.
page 12 parsing completed.
page 13 parsing completed.
page 14 parsing completed.
page 15 parsing completed.
page 16 parsing completed.
page 17 parsing completed.
900000
Json file saving completed.


In [59]:
from itertools import compress
import pandas as pd
from operator import itemgetter
from itertools import groupby
#功能：list里面的每一个元素都是dict，根据dict某一个key进行去重
#函数1
def distinct(items,key):
    key = itemgetter(key)
    items = sorted(items, key=key)
    return [next(v) for _, v in groupby(items, key=key)]
#函数2
#def distinct(items,key):  
#    mask = (~pd.Series(map(itemgetter(key), items)).duplicated()).tolist()
#    return list(compress(items, mask))
with open("/data/fjs/data/math/math.json",'r') as load_f:
    items = json.load(load_f)
    print (len(items))#去重前条目
    items =distinct(items,'quid')#去重key=quid
    print (len(items))#去重后条目

900000
900000
